#   [Py4Hw User Guide](../UserGuide.ipynb)> 1.6 Wires

Wires are required to interconnect the logic blocks instantiated in circuits.
In py4hw wires are created by instantiating objects of the **Wire** class.

Wire require the following parameters in their constructor

1. The parent logic circuit. Every wire is assigned to a logic circuit.

2. A name. All objects in py4hw have a name.

3. A bit width. Wires can have a single bit, or an arbitrary number of bits. There is no limit to the number of bits.



In [10]:
import py4hw

In [11]:
sys = py4hw.HWSystem()

In [12]:
a = py4hw.Wire(sys, 'a', 2)
b = py4hw.Wire(sys, 'b', 2)